In [ ]:
# !pip3 install requests
# !pip3 install python-dotenv
# !pip3 install gspread
# !pip3 install oauth2client
# !pip3 install google-api-python-client

In [ ]:
import os
import requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [ ]:
load_dotenv('.env')

# Retrieve Canvas Inbox starred messages (conversations)

In [ ]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_URL')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Parameters to control pagination
params = {
    'per_page': 50  # Adjust this number as needed, up to the maximum allowed by the API
}

In [ ]:
conversations_response = requests.get(f'{base_url}/conversations', headers=headers, params=params)
conversations = conversations_response.json()

# List to store all messages with sender names
sender_data = []

# Get sender names from starred conversations and set status to False
for conversation in conversations:
    if conversation["starred"]:
        sender_name = conversation["participants"][0]["name"]
        sender_data.append([sender_name])  # Appending [sender_name, False] to sender_data

print("Number of starred conversations: ", len(sender_data))
print(sender_data)  # Print sender data for testing

# Save result into Google Sheet

In [ ]:
def append_checkbox(creds, spreadsheet, worksheet, total_rows):

    # Create a service object for the Google Sheets API
    service = build('sheets', 'v4', credentials=creds)

    # Update the header of the checkbox column to "Replied?"
    worksheet.update_cell(1, 2, 'Replied?')  # Assuming the checkbox column is column B (index 2)

    # Add checkboxes to the column
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body={
        'requests': [
            {
                'repeatCell': {
                    'cell': {
                        'dataValidation': {
                            'condition': {'type': 'BOOLEAN'},
                            'showCustomUi': True
                        }
                    },
                    'range': {
                        'sheetId': worksheet.id,
                        'startRowIndex': 1,  # Skip the header row
                        'endRowIndex': total_rows + 1,
                        'startColumnIndex': 1,  # Column B
                        'endColumnIndex': 2
                    },
                    'fields': 'dataValidation'
                }
            }
        ]
    }).execute()

In [ ]:
def append_data(sender_data, worksheet, spreadsheet, creds):

    # Collect all existing sender data from all other worksheets
    all_existing_senders = set()
    for sheet in spreadsheet.worksheets():
        if sheet.title != worksheet.title:
            existing_data = sheet.get_all_values()
            existing_senders = [row[0] for row in existing_data if row]  # Assuming sender names are in the first column
            all_existing_senders.update(existing_senders)

    # Write column headers if the sheet is empty
    if worksheet.get_all_values():
        worksheet.append_row(['Sender Name'])

    # Write sender data to the worksheet
    num_breaker = 0
    num_added = 0
    total_line = 0
    num_to_add = len(sender_data)
    person_to_assign = 7
    breaker_row = ['******* Finished my part? *******']
    for sender in sender_data:
        if sender[0] not in all_existing_senders:
            worksheet.append_row(sender)  # Writing sender data
            print("Added", sender[0])
            num_added += 1
            total_line += 1
            # Add the breaker row after every 7 rows
            if (person_to_assign > 0) and (num_added % round(num_to_add / person_to_assign)) == 0:
                worksheet.append_row(breaker_row)
                print("Added breaker row **********")
                num_to_add -= round(num_to_add / person_to_assign)
                person_to_assign -= 1
                num_breaker += 1
                total_line += 1
        else:
            print(f"Skipped {sender[0]} (already exists)")

    print("Total breaker row(s) added:", num_breaker)

    # Set the width of the first column
    service = build('sheets', 'v4', credentials=creds)
    request = {
        'requests': [
            {
                'updateDimensionProperties': {
                    'range': {
                        'sheetId': worksheet.id,
                        'dimension': 'COLUMNS',
                        'startIndex': 0,
                        'endIndex': 1
                    },
                    'properties': {
                        'pixelSize': 200  # Set the desired width in pixels
                    },
                    'fields': 'pixelSize'
                }
            }
        ]
    }
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body=request).execute()

    if total_line > 1:
        append_checkbox(creds, spreadsheet, worksheet, total_line)

    print("Total name(s) added:", num_added)

In [ ]:
# Authenticate with Google Sheets API
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"
         ]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Define the folder ID where you want to save the Google Sheets file
folder_id = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
# Define file name for the Google Sheets file
file_name = os.getenv('GOOGLE_SHEET_FILE_NAME')

# Get today's date and format it
today_date = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
# Check if a file with the same name already exists
file_exists = False
for file in client.list_spreadsheet_files():
    if file['name'] == file_name:
        print("Same-name Google Sheet file already exists in the folder...")
        file_exists = True
        spreadsheet = client.open(file_name)
        break

# If the file does not exist, create a new one and write sender data to it
if not file_exists:
    print(f'Creating a new Google Sheet file named "{file_name}"...')
    spreadsheet = client.create(file_name, folder_id=folder_id)

    # Rename the default worksheet ('Sheet1') with today's date
    worksheet = spreadsheet.sheet1
    worksheet.update_title(today_date)

    append_data(sender_data, worksheet, spreadsheet, creds)

else:
    print(f'Creating a new worksheet in "{file_name}"...')
    # Add a new worksheet with today's date as the title and write sender data to it
    worksheet = spreadsheet.add_worksheet(title="3", rows="100", cols="20")  # Create a new worksheet

    append_data(sender_data, worksheet, spreadsheet, creds)


There is a bug with Google Drive. Even though I have deleted the Google Sheet file from the Google Drive UI, it still shows up in this programming interface. So, I have to delete them using the programming interface to ensure that the file is deleted.

In [ ]:
response = client.list_spreadsheet_files()

# Delete each spreadsheet file
for file in response:
    print(f"Deleting file: {file['name']} ({file['id']})")
    client.del_spreadsheet(file['id'])